# SIE Proxy Correlation

This script is for producing heatmaps to show the correlation between SIE and the proxies within the _Proxy_combined_data_v4_ dataset.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from scipy import stats
from datetime import datetime

Create an array of 3-month season acronyms.

In [ ]:
seasons = ['OND','NDJ','DJF','JFM','FMA','MAM','AMJ','MJJ','JJA','JAS','ASO','SON','OND']

Read in the data.

In [ ]:
folder = '~/Desktop/IMAS/ncfiles/'  #filepath
pfile = 'Proxy_combined_data_v4.nc'  #proxy filename
pvar = 'SIE'  #proxy sea ice extent variable name
pdata = xr.open_dataset(folder+pfile) #open proxy as Dataset

Initialise an empty array to store the correlations.

In [ ]:
corrs = np.zeros([12,36])

Fix the start year. Then create arrays of the proxy names and respective end years.

In [ ]:
proxies = ['AP_stacked_MSA', 'Law_Dome_MSA', 'Ferrigno_MSA','WHG_dex','SOFI_formation','SOFI_breakout','SOFI_duration']
endyears = [2002,1995,2010,2006,2008,2008,2008]

Now loop through the seven proxies, logging a correlation value at each season/longitude intersection before plotting the heatmap.

In [ ]:
for i in range(7):
    
    proxy = proxies[i]
    endyear = endyears[i]
    
    if proxy=='WHG_dex':
        pts = pdata[proxy].sel(year = slice(startyear,endyear))
        pts = pts.mean('month')
    else:
        pts = pdata[proxy].sel(year = slice(startyear,endyear)) 

    
    #Creating time series over months
    for m in range(12): #loop over the months
        m+=1
        months = [(m-3)%12+1, (m-2)%12+1, (m-1)%12+1] #nonneg month ints for season ending in m
        monat = pdata[pvar].sel(year = slice(startyear,endyear), month=months) 
        monat = monat.mean('month')
        for l in np.arange(0.25,360,10): #loop over the longitudes
            sie = monat.sel(lon= slice(l, l+10)) 
            sie = sie.mean(dim='lon')
            corrs[m-1,int((l-0.25)/10)] = stats.pearsonr(pts,sie)[0] #Place corcoef in corrs
    
    ax = plt.subplot()
    
    clev = [-0.8,-0.7,-0.6,-0.5,-0.4,0.4,0.5,0.6,0.7,0.8]  #contour levels
    #clev = np.arange(-1, 1.1, 0.1)
    corrsxr = xr.DataArray(corrs)
    corrsxr.plot.pcolormesh(levels = clev)
    
    ax.set_title(proxy)
    ax.set_ylabel('Season')
    #plt.yticks(ticks=np.arange(12))
    ax.set_xlabel('Longitude')
    ax.set_xticks(ticks=np.arange(3,36,3), labels=['','60E','','120E','','180','','120W','','60W',''])
    ax.set_yticks(ticks=np.arange(-0.5,12,1), labels=seasons, rotation=0)
    
    plt.show()

__Note on seasonality:__ Pearson correlation of a time series for a given month will consider effect of annual changes. We will not see the effect of seasonality as correlation coefficients of months are calculated independently.